## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('Data/Real_Estate_Sales_2001-2022_GL.csv', low_memory=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097629 entries, 0 to 1097628
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1097629 non-null  int64  
 1   List Year         1097629 non-null  int64  
 2   Date Recorded     1097627 non-null  object 
 3   Town              1097629 non-null  object 
 4   Address           1097578 non-null  object 
 5   Assessed Value    1097629 non-null  float64
 6   Sale Amount       1097629 non-null  float64
 7   Sales Ratio       1097629 non-null  float64
 8   Property Type     715183 non-null   object 
 9   Residential Type  699240 non-null   object 
 10  Non Use Code      313451 non-null   object 
 11  Assessor Remarks  171228 non-null   object 
 12  OPM remarks       13031 non-null    object 
 13  Location          298111 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 117.2+ MB


In [3]:
data.duplicated().sum() # no duplicate rows!

0

In [4]:
d1 = data.drop(columns=['Serial Number', 'Assessor Remarks', 'OPM remarks', 'Non Use Code']) # remove some unnecessary columns
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097629 entries, 0 to 1097628
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   List Year         1097629 non-null  int64  
 1   Date Recorded     1097627 non-null  object 
 2   Town              1097629 non-null  object 
 3   Address           1097578 non-null  object 
 4   Assessed Value    1097629 non-null  float64
 5   Sale Amount       1097629 non-null  float64
 6   Sales Ratio       1097629 non-null  float64
 7   Property Type     715183 non-null   object 
 8   Residential Type  699240 non-null   object 
 9   Location          298111 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 83.7+ MB


## Property Types

In [5]:
print(d1['Property Type'].value_counts())
print()
print(d1['Residential Type'].value_counts())
print()
print(d1[['Residential Type', 'Property Type']])
# many rows have null entires for property type/residential type

Property Type
Single Family     401612
Residential       151064
Condo             105420
Two Family         26408
Three Family       12586
Vacant Land         7824
Commercial          5987
Four Family         2150
Apartments          1327
Industrial           795
Public Utility        10
Name: count, dtype: int64

Residential Type
Single Family    508044
Condo            136822
Two Family        34848
Three Family      16567
Four Family        2959
Name: count, dtype: int64

        Residential Type  Property Type
0          Single Family    Residential
1                    NaN     Commercial
2          Single Family    Residential
3          Single Family    Residential
4                  Condo    Residential
...                  ...            ...
1097624    Single Family  Single Family
1097625    Single Family  Single Family
1097626    Single Family  Single Family
1097627    Single Family  Single Family
1097628              NaN            NaN

[1097629 rows x 2 columns]


In [6]:
# Consolidate property type information
type_df = d1
type_df['Residential Type'] = type_df['Residential Type'].fillna('') # for now I will replace the null values with empty strings so that I can more easily manipulate the strings
type_df['Property Type'] = type_df['Property Type'].fillna('')
type_df['Type'] = (type_df['Property Type'] + ' ' + type_df['Residential Type'])
type_df['Type'] = type_df['Type'].str.strip()
print(type_df['Type'].unique())
print()
type_df['Type'] = type_df['Type'].replace(r'(\w+ Family) \1', r'Residential \1', regex=True).replace('Condo Condo', 'Condo')
print(type_df['Type'].unique())
print()
type_df = type_df.drop(columns=['Residential Type', 'Property Type'])
type_df['Type'] = type_df['Type'].replace('', np.nan) # change the empty strings back to null values
print(type_df.info())

['Residential Single Family' 'Commercial' 'Residential Condo'
 'Residential Two Family' 'Vacant Land' '' 'Apartments' 'Industrial'
 'Residential Four Family' 'Residential Three Family' 'Public Utility'
 'Condo Condo' 'Two Family Two Family' 'Three Family Three Family'
 'Single Family Single Family' 'Four Family Four Family']

['Residential Single Family' 'Commercial' 'Residential Condo'
 'Residential Two Family' 'Vacant Land' '' 'Apartments' 'Industrial'
 'Residential Four Family' 'Residential Three Family' 'Public Utility'
 'Condo']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097629 entries, 0 to 1097628
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   List Year       1097629 non-null  int64  
 1   Date Recorded   1097627 non-null  object 
 2   Town            1097629 non-null  object 
 3   Address         1097578 non-null  object 
 4   Assessed Value  1097629 non-null  float64
 5   Sale Amount  

## Locations

In [7]:
print(type_df['Location'].info())
print()
print(type_df['Location'].unique())
print()
# note that many locations are null

<class 'pandas.core.series.Series'>
RangeIndex: 1097629 entries, 0 to 1097628
Series name: Location
Non-Null Count   Dtype 
--------------   ----- 
298111 non-null  object
dtypes: object(1)
memory usage: 8.4+ MB
None

['POINT (-72.343628962 41.728431984)' nan
 'POINT (-72.846365959 41.781677018)' ... 'POINT (-72.80758 41.68147)'
 'POINT (-72.96622 41.32883)' 'POINT (-72.07006 41.53315)']



In [8]:
# I will just look at the ones that have locations
location_df = type_df[type_df['Location'].notna()].copy()
location_df['Latitude'] = location_df['Location'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)')[1]
location_df['Longitude'] = location_df['Location'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)')[0]
location_df = location_df.drop(columns=['Location'])
location_df['Latitude'] = location_df['Latitude'].astype(float)
location_df['Longitude'] = location_df['Longitude'].astype(float)
print(location_df.info()) # there are also some rows without addresses
print()
location_df = location_df[location_df['Address'].notna()]
print(location_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 298111 entries, 0 to 1097622
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   List Year       298111 non-null  int64  
 1   Date Recorded   298111 non-null  object 
 2   Town            298111 non-null  object 
 3   Address         298106 non-null  object 
 4   Assessed Value  298111 non-null  float64
 5   Sale Amount     298111 non-null  float64
 6   Sales Ratio     298111 non-null  float64
 7   Type            226458 non-null  object 
 8   Latitude        298111 non-null  float64
 9   Longitude       298111 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 25.0+ MB
None

<class 'pandas.core.frame.DataFrame'>
Index: 298106 entries, 0 to 1097622
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   List Year       298106 non-null  int64  
 1   Date Recorded   298106

In [9]:
import folium

# Create a map centered around a specific latitude and longitude
map_center = [41.7658, -72.6734]  # Example: Coordinates for Hartford, Connecticut
mymap = folium.Map(location=map_center, zoom_start=12)

count = 500
lats = location_df['Latitude'].tolist()[:count]
longs = location_df['Longitude'].tolist()[:count]
values = location_df['Assessed Value'].tolist()[:count]
prices = location_df['Sale Amount'].tolist()[:count]
addresses = location_df['Address'].tolist()[:count]
types = location_df['Type'].tolist()[:count]

for lat, long, val, price, t, add in zip(lats,longs,values,prices, types, addresses):
    folium.Marker([lat, long], popup=(t, add, val, price)).add_to(mymap)

mymap
# when looking at this map I notice that many locations sell for about double the assessed value, this could be something to look into

# Export

In [12]:
cleaned_df = location_df.dropna(subset=['Type'])
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226458 entries, 0 to 1097622
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   List Year       226458 non-null  int64  
 1   Date Recorded   226458 non-null  object 
 2   Town            226458 non-null  object 
 3   Address         226458 non-null  object 
 4   Assessed Value  226458 non-null  float64
 5   Sale Amount     226458 non-null  float64
 6   Sales Ratio     226458 non-null  float64
 7   Type            226458 non-null  object 
 8   Latitude        226458 non-null  float64
 9   Longitude       226458 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 19.0+ MB


In [13]:
cleaned_df.to_csv('Data/cleaned_data.csv', index=False)